## Transfer Learning for Monkey Species Classification

In this notebook we are going to implement transfer learning to classify 10 monkey species. The dataset collected is from Kaggle and can be obtained via this link : https://www.kaggle.com/slothkong/10-monkey-species?select=monkey_labels.txt

Before everything its always best to import the necessary packages and then move ahead.

In [1]:
## Importing necessary packages ##

import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import mobilenet_v2
import torchvision.transforms as transforms
from torchvision.utils import make_grid

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Done!

All the necessary packages are imported.

Now lets move ahead and import out dataset by setting some transformations.

Now since we are going to use a pretrained network we need to polish out the input following the scope of the input of the network.

Now, Pytorch made an excellent documentation (https://pytorch.org/vision/0.8/models.html) regarding this and I have gone through it and have summarized the transformation as follows for any network.

1. Image Size >= 224  -- We are going to take 224 for our model
2. Normalized with a mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]

Thats all.

So lets do that quickly and carry on!!

In [2]:
## Setting the transforms ##

trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406] , std = [0.229, 0.224, 0.225])
])

Boom!! 
All the transformations are set.

Lets jump in and import our dataset.

In [3]:
## Loading our data ##

monkey_data = ImageFolder(root = 'training' , 
                          transform = trans)

Lets check!!

In [19]:
## Checking our dataset ##

print('Number of Images :' , len(monkey_data))

print('Image Size :' , monkey_data[0][0].shape)

print('Label :' , monkey_data[0][1])

print('Maximum Value :' , torch.max(monkey_data[0][0]))

print('Minimum Value :' , torch.min(monkey_data[0][0]))

Number of Images : 1097
Image Size : torch.Size([3, 224, 224])
Label : 0
Maximum Value : tensor(2.6400)
Minimum Value : tensor(-2.1179)


All nicely set.

Now, lets set our dataloader.

In [5]:
## Setting the Dataloader ##

monkey_dataloader = DataLoader(monkey_data,
                               batch_size = 64,
                               shuffle = True)

Lets check the size of the dataloader and then move ahead to putting it to a GPU to make our work more efficient.

In [6]:
## Checking the Datalaoder ##

print('Number of batches :' ,len(monkey_dataloader))

Number of batches : 18


So, the dataloader will pop out 18 batches, since 18 * 64 = 1152 which includes 1097 images in our training dataset.

With that aside, lets put our dataloader in the GPU.

But for that we need to define some GPU utility functions. 

So, lets go ahead and implement that first.

In [7]:
## GPU utility functions ##

def check_cuda():
    if torch.cuda.is_available():
        return torch.device('cuda')
    return torch.device('cpu')

def transfer_data(data , device):
    if isinstance(data , (tuple , list)):
        return [transfer_data(each_data , device) for each_data in data]
    return data.to(device)

Now, lets create a Dataloader class that yields data to the GPU.

In [8]:
## GPU Datalaoder ##

class gpu_dataloader():
    def __init__(self , dl , device):
        self.dl = dl
        self.device = device
    
    def __iter__(self):
        for batch in self.dl:
            yield transfer_data(batch , self.device)
    
    def __len__(self):
        return len(self.dl)

With that lets make our own dataloader as GPU Dataloader.

In [9]:
## Setting default device ##

device = check_cuda()

## Initializing GPU dataloader ##

monkey_dl = gpu_dataloader(monkey_dataloader , device)

And done.

In [10]:
## Checking the GPU dataloader ##

print('Length of the GPU dataloader:' , len(monkey_dl))

Length of the GPU dataloader: 18


And it matches!!

Carrying on.

In this notebook we are going to implement Transfer Learning through mobile net.

So, lets first fetch our model with pretrained weights.

In [11]:
## Getting our base model ##

base_model = mobilenet_v2(pretrained = True)

Boom!

The pretrained model is loaded.

Lets check it

In [12]:
## Printing the base model ##

print(base_model)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

Now with this pretrained model, we need to do two things.

1. Freeze all the layers except the batch normalization ones, since they need to alter the gamma and beta value for the working dataset.

2. Change the final layers for the classification based on our dataset.

In [13]:
## Freezing our model ##

for name , param in base_model.named_parameters():
    if name == 'BatchNorm2d':
        continue
    param.requires_grad = False

And done!!

Now change our model's final layer.

In [14]:
## Changing our model top ##

base_model.classifier = nn.Sequential(
    nn.Linear(1280 , 500),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(500 , 10)
)

## print the model ##

print(base_model)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

Yes!! The last layers are changed.

Now lets set our training loop with the loss function , optimizers and also a generic accuracy utility function.

In [15]:
## Defining accuracy ##

def accuracy(pred , target):
    actual = torch.argmax(pred , dim = 1)
    num = (torch.sum(actual == target)).item()
    den = pred.numel()
    return num / den

With that out of the way.

Lets set our loss function and optimizer.

In [16]:
## Setting Loss function ##

criterion = nn.functional.cross_entropy

## Setting Optimizer ##

optim = torch.optim.Adam(base_model.parameters() , lr = 0.001)

Now lets set our training loop.

In [17]:
## Putting the model into GPU##

base_model = base_model.to('cuda')

## Training ##

num_epoch = 10

for epoch in range(num_epoch):
    
    cum_acc = []
    cum_loss = []
    
    for img , label in monkey_dl:
        
        acc = 0
        loss =0
        
        pred = base_model(img)
        
        optim.zero_grad()
        
        loss = criterion(pred , label)
        
        cum_loss.append(loss.item())
        
        acc = accuracy(pred , label)
        
        #print(acc_pred)
        #print(label)
        
        cum_acc.append(acc)
        
        loss.backward()
        
        optim.step()
        
    #print(cum_acc)
    #print(cum_loss)
    
    #epoch_acc = sum(cum_acc) / len(cum_acc)
    
    #epoch_loss = sum(cum_loss) / len(cum_loss)
    
    print('Epochs : {} / {} --> Accuracy = {:.3f} , Loss = {:.3f}'.format(epoch + 1 , num_epoch , acc , loss.item()))

Epochs : 1 / 10 --> Accuracy = 0.078 , Loss = 0.805
Epochs : 2 / 10 --> Accuracy = 0.089 , Loss = 0.483
Epochs : 3 / 10 --> Accuracy = 0.100 , Loss = 0.148
Epochs : 4 / 10 --> Accuracy = 0.100 , Loss = 0.095
Epochs : 5 / 10 --> Accuracy = 0.100 , Loss = 0.014
Epochs : 6 / 10 --> Accuracy = 0.100 , Loss = 0.146
Epochs : 7 / 10 --> Accuracy = 0.100 , Loss = 0.054
Epochs : 8 / 10 --> Accuracy = 0.100 , Loss = 0.110
Epochs : 9 / 10 --> Accuracy = 0.100 , Loss = 0.101
Epochs : 10 / 10 --> Accuracy = 0.100 , Loss = 0.019


Our model didn't perform better. Just 10% accuracy.